In [1]:
%matplotlib notebook
import importlib
from config import Player, EnvConfig
from environments.flipit_utils import Action, ActionTargetPair, PlayerTargetPair
from environments.poachers_renderer import Renderer
import environments.poachers
environments.poachers = importlib.reload(environments.poachers)
import torch
from torchrl.envs.utils import check_env_specs
from torchrl.envs import Compose, TransformedEnv, RewardSum, Stack

NUM_NODES = 20
NUM_STEPS = 40
SEED = 41

config = EnvConfig(
    num_nodes=NUM_NODES,
    num_steps=NUM_STEPS,
    seed=SEED,
    env_name="poachers",
)

In [2]:
env_map = environments.poachers.PoachersMap(config, "cpu")

In [3]:
env = environments.poachers.PoachersEnv(config, env_map, "cpu")

check_env_specs(env)
out = env.reset()

ValueError: Invalid action detected: tensor([4, 5], dtype=torch.int32). Cannot prepare node 12.

In [ ]:
renderer = Renderer(env, SEED)
renderer.render()

In [ ]:
from algorithms.generic_policy import MultiAgentPolicy, CombinedPolicy
from algorithms.generator import AgentGenerator
from algorithms.keys_processors import CombinedExtractor
from config import AgentNNConfig, BackboneConfig, HeadConfig, EnvConfig
import yaml

from config import TrainingConfig, LossConfig
from algorithms.simple_nn import TrainableNNAgentPolicy

with open("configs/run/test_single_training_poachers.yaml", "r") as file:
    config = yaml.safe_load(file)

env_config = EnvConfig.from_dict(config)
training_config_attacker = TrainingConfig.from_dict(config, suffix=f"_attacker")
loss_config_attacker = LossConfig.from_dict(config, suffix=f"_attacker")
agent_config = AgentNNConfig.from_dict(config)
backbone_config = BackboneConfig.from_dict(config, suffix=f"_backbone")
head_config = HeadConfig.from_dict(config, suffix=f"_head")


# attacker_agent = MultiAgentPolicy(
#     action_size=env.base_env.action_size,
#     player_type=1,
#     device="cpu",
#     embedding_size=32,
#     run_name="test",
#     policy_generator=AgentGenerator(
#         TrainableNNAgentPolicy,
#         {
#             "action_size": env.base_env.action_size,
#             "total_steps": env.base_env.num_steps,
#             "player_type": 1,
#             "embedding_size": 32,
#             "device": "cpu",
#             "loss_config": loss_config_attacker,
#             "training_config": training_config_attacker,
#             "run_name": "test",
#             "use_transformer": True,
#         }
#     ),
# )
attacker_extractor = CombinedExtractor(player_type=1, env=env, actions=backbone_config.extractors)
attacker_agent = TrainableNNAgentPolicy(
    player_type=1,
    max_sequence_size=NUM_STEPS + 1,
    extractor=attacker_extractor,
    action_size=env.action_size,
    env_type=env_config.env_pair,
    device="cpu",
    loss_config=loss_config_attacker,
    training_config=training_config_attacker,
    run_name="test",
    backbone_config=backbone_config,
    head_config=head_config,
    agent_config=agent_config,
)


attacker_agent.eval()
attacker_agent.load("saved_models/2025-07-07_23:54:35-attacker-/attacker/agent_0.pth")

In [ ]:
from algorithms.generic_policy import RandomAgent, GreedyOraclePoacherAgent


greedy_agent = GreedyOraclePoacherAgent(
    action_size=env.action_size,
    player_type=1,
    device="cpu",
    run_name="test",
    total_steps=NUM_STEPS,
    embedding_size=agent_config.embedding_size,
    env_map=env_map,
)

random_agent = RandomAgent(
    action_size=env.action_size,
    player_type=1,
    device="cpu",
    run_name="test",
    embedding_size=agent_config.embedding_size,
)

In [ ]:
greedy_agent(out)["action"]

In [ ]:
random_agent(out)["action"]

In [ ]:
a = attacker_agent(out)
a["action"]

In [ ]:
a["logits"]

In [ ]:
out.update({
    "action": torch.tensor([4, 5]),
})
out2 = env.step(out)

In [ ]:
renderer.render()
out = out2["next"]
out["game_id"] = out2["game_id"]

In [ ]:
out["done"]

In [ ]:
out2["next"]["available_moves"]

In [ ]:
out2["next"]["node_reward_info"]

In [ ]:
renderer.render()

In [ ]:
out = out2["next"]

In [ ]:
env.

In [4]:
import yaml
from algorithms.simple_nn import TrainableNNAgentPolicy, NNAgentPolicy
from algorithms.generic_policy import CombinedPolicy, MultiAgentPolicy, ExplorerAgent
from algorithms.keys_processors import CombinedExtractor
from algorithms.generator import AgentGenerator
from config import TrainingConfig, LossConfig, EnvConfig, Player, AgentNNConfig, BackboneConfig, HeadConfig

device = torch.device("cuda:0")

with open("configs/run/test_single_training_poachers.yaml", "r") as file:
    config = yaml.safe_load(file)

run_name = "test"

env_config_ = EnvConfig.from_dict(config)
training_config = TrainingConfig.from_dict(config, suffix=f"_attacker")
loss_config = LossConfig.from_dict(config, suffix=f"_attacker")
agent_config = AgentNNConfig.from_dict(config)
backbone_config = BackboneConfig.from_dict(config, suffix=f"_backbone")
head_config = HeadConfig.from_dict(config, suffix=f"_head")

# env_map, env = env_config_.create("cpu")

defender_extractor = CombinedExtractor(player_type=0, env=env, actions_map=backbone_config.extractors)
attacker_extractor = CombinedExtractor(player_type=1, env=env, actions_map=backbone_config.extractors)

defender_agent = NNAgentPolicy(
    player_type=0,
    max_sequence_size=env_config_.num_steps + 1,
    extractor=defender_extractor,
    action_size=env.action_size,
    backbone_config=backbone_config,
    head_config=head_config,
    agent_config=agent_config,
    device=device,
    run_name=run_name,
)
attacker_agent = TrainableNNAgentPolicy(
    player_type=1,
    max_sequence_size=env_config_.num_steps + 1,
    extractor=attacker_extractor,
    action_size=env.action_size,
    # env_type=env_config_.env_pair,
    device=device,
    loss_config=loss_config,
    training_config=training_config,
    run_name=run_name,
    backbone_config=backbone_config,
    head_config=head_config,
    agent_config=agent_config,
    #scheduler_steps=training_config.total_steps_per_turn // training_config.steps_per_batch + 5,
)

exploration_defender = ExplorerAgent(
    action_size=env.action_size,
    player_type=0,
    device=device,
    run_name=run_name,
    total_steps=env.num_steps,
    embedding_size=agent_config.embedding_size,
)
exploration_attacker = ExplorerAgent(
    action_size=env.action_size,
    player_type=1,
    device=device,
    run_name=run_name,
    total_steps=env.num_steps,
    embedding_size=agent_config.embedding_size,
)
combined_policy = CombinedPolicy(
    defender_agent,
    attacker_agent,
    #exploration_defender=exploration_defender,
    #exploration_attacker=exploration_attacker,
).to("cuda")

In [5]:
training_config.steps_per_batch

70

In [6]:
from torchrl.collectors import SyncDataCollector, MultiaSyncDataCollector, MultiSyncDataCollector
from torchrl.envs import ParallelEnv
torch.multiprocessing.set_start_method('spawn', force=True)
torch.multiprocessing.set_sharing_strategy('file_system')

collector = SyncDataCollector(
    #[env.create_from_self for _ in range(4)],
    # env.create_from_self,
    ParallelEnv(4, env.create_from_self),
    policy=combined_policy,
    frames_per_batch=3000,
    total_frames=3000,
    split_trajs=False,
    device="cpu",
    policy_device="cuda:0",
)

In [7]:
import time
start = time.time()
tensordict = next(collector.iterator())
print(time.time() - start)

11.016482830047607


In [15]:
tensordict[0]["position_seq"]

tensor([[[-1, -1, -1,  ..., -1, -1,  1],
         [-1, -1, -1,  ..., -1, -1, 12]],

        [[-1, -1, -1,  ..., -1,  1,  0],
         [-1, -1, -1,  ..., -1, 12, 14]],

        [[-1, -1, -1,  ...,  1,  0,  1],
         [-1, -1, -1,  ..., 12, 14, 12]],

        ...,

        [[-1, -1, -1,  ..., -1, 17, 10],
         [-1, -1, -1,  ..., -1, 12, 13]],

        [[-1, -1, -1,  ..., 17, 10, 12],
         [-1, -1, -1,  ..., 12, 13, 14]],

        [[-1, -1, -1,  ..., 10, 12, 10],
         [-1, -1, -1,  ..., 13, 14, 15]]])

In [ ]:
tensordict["next"]["reward"][:, 1].mean()

In [23]:
tensordict["next"]["reward"][:, 1].max()

NameError: name 'tensordict' is not defined

In [ ]:
from torch_geometric.loader import DataLoader

loader = DataLoader([env_map, env_map], batch_size=2)

batch = next(iter(loader))

In [24]:
batch

NameError: name 'batch' is not defined

In [ ]:
batch.edge_index.shape

In [25]:
from unittest.mock import patch
from torch import nn

In [26]:
class _Linear(nn.Linear):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        bias: bool = True,
        weight_initializer: str | None = None,
        bias_initializer: str | None = None,
    ):
        super().__init__(
            in_features=in_channels,
            out_features=out_channels,
            bias=bias,
        )

In [27]:
with patch("torch_geometric.nn.conv.gcn_conv.Linear", side_effect=_Linear):
    from torch_geometric.nn.conv import GCNConv
    convs = nn.ModuleList([
        GCNConv(2, 5, aggr="mean"),
        GCNConv(5, 5, aggr="mean"),
        GCNConv(5, 5, aggr="mean"),
    ])

In [28]:
x = batch.x
for conv_layer in convs:
    x = conv_layer(x, batch.edge_index)
    x = torch.relu(x)
x

NameError: name 'batch' is not defined

In [ ]:
from algorithms.generic_policy import MultiAgentPolicy, CombinedPolicy
from algorithms.generator import AgentGenerator
from algorithms.keys_processors import CombinedExtractor
from config import AgentNNConfig, BackboneConfig, HeadConfig, EnvConfig
import yaml

from config import TrainingConfig, LossConfig
from algorithms.simple_nn import TrainableNNAgentPolicy

with open("configs/run/test_single_training_poachers.yaml", "r") as file:
    config = yaml.safe_load(file)

env_config = EnvConfig.from_dict(config)
training_config_attacker = TrainingConfig.from_dict(config, suffix=f"_attacker")
loss_config_attacker = LossConfig.from_dict(config, suffix=f"_attacker")
agent_config = AgentNNConfig.from_dict(config)
backbone_config = BackboneConfig.from_dict(config, suffix=f"_backbone")
head_config = HeadConfig.from_dict(config, suffix=f"_head")


# attacker_agent = MultiAgentPolicy(
#     action_size=env.base_env.action_size,
#     player_type=1,
#     device="cpu",
#     embedding_size=32,
#     run_name="test",
#     policy_generator=AgentGenerator(
#         TrainableNNAgentPolicy,
#         {
#             "action_size": env.base_env.action_size,
#             "total_steps": env.base_env.num_steps,
#             "player_type": 1,
#             "embedding_size": 32,
#             "device": "cpu",
#             "loss_config": loss_config_attacker,
#             "training_config": training_config_attacker,
#             "run_name": "test",
#             "use_transformer": True,
#         }
#     ),
# )
attacker_extractor = CombinedExtractor(player_type=1, env=env, actions_map=backbone_config.extractors)
attacker_agent = TrainableNNAgentPolicy(
    player_type=1,
    max_sequence_size=NUM_STEPS + 1,
    extractor=attacker_extractor,
    action_size=env.action_size,
    env_type=env_config.env_pair,
    device="cpu",
    loss_config=loss_config_attacker,
    training_config=training_config_attacker,
    run_name="test",
    backbone_config=backbone_config,
    head_config=head_config,
    agent_config=agent_config,
)

In [ ]:
from utils import create_replay_buffer

In [ ]:
replay_buffer = create_replay_buffer(training_config_attacker)

In [29]:
env_map.move_cost * (NUM_STEPS - 2)

tensor(-0.1057)

In [30]:
env_map.x[0].sum()

tensor(0.5201)

In [31]:
env_map.x

tensor([[ 0.6772, -0.1571],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.5329, -0.0349],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.8392, -0.9298],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.6984, -0.4183]])

In [32]:
env_map.preparation_reward

tensor(1.9236)